In [1]:
import pandas as pd
from sklearn.datasets import make_classification
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeClassifier
from numpy import argmax, array
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from scipy.linalg import pinv2


In [8]:
train = pd.read_csv("kdd_train.csv")
test = pd.read_csv("kdd_test.csv")

print(test.shape,train.shape)
print(test['duration'])

(22544, 42) (125973, 42)
0            5
1            0
2            0
3         1979
4        14462
         ...  
22539        0
22540        0
22541        0
22542        0
22543        0
Name: duration, Length: 22544, dtype: int64


In [43]:
for i in ['protocol_type', 'service', "flag", "labels"]:
    values_train = array(train[i])
    values_test = array(test[i])
    label_encoder = LabelEncoder()
    integer_encoded_train = label_encoder.fit_transform(values_train)
    integer_encoded_test = label_encoder.fit_transform(values_test)
    train[i] = integer_encoded_train
    test[i] = integer_encoded_test

X_train = train.iloc[:, :41]
y_train = train.iloc[:, 41]

X_test = test.iloc[:, :41]
y_test = test.iloc[:, 41]

#print(X_train.head(),y_train.head())
print(X_test.head(),y_test.head())
#print(list(y_train))

duration  protocol_type  service  flag  src_bytes  dst_bytes  land  \
0         5              1       51     9       2429        475     0   
1         0              2       12     9         45        134     0   
2         0              2       12     9         45         80     0   
3      1979              2       41     9        145        105     0   
4     14462              1       41     4          1          0     0   

   wrong_fragment  urgent  hot  ...  dst_host_count  dst_host_srv_count  \
0               0       0    0  ...             180                  57   
1               0       0    0  ...             194                 155   
2               0       0    0  ...             255                 255   
3               0       0    0  ...             255                   2   
4               0       0    0  ...             255                   2   

   dst_host_same_srv_rate  dst_host_diff_srv_rate  \
0                    0.11                    0.02   
1      

In [42]:

rfe = RFE(estimator=DecisionTreeClassifier(), n_features_to_select=20)
# fit RFE
rfe.fit(X_train, y_train)
# summarize all features
rfe_X_train = pd.DataFrame()
rfe_X_test = pd.DataFrame()
for i in range(X_train.shape[1]):
    print('Column: %d, Selected %s, Rank: %.3f' % (i, rfe.support_[i], rfe.ranking_[i]))
    col_name_train = train[train.columns[i]]
    col_name_test = test[test.columns[i]]
    if rfe.support_[i]:
        rfe_X_train = pd.concat([rfe_X_train, col_name_train], axis = 1)
        rfe_X_test = pd.concat([rfe_X_test, col_name_test], axis = 1)
      
print (rfe_X_train.shape, rfe_X_test.head())
print(col_name_train)


Column: 0, Selected False, Rank: 2.000
Column: 1, Selected True, Rank: 1.000
Column: 2, Selected True, Rank: 1.000
Column: 3, Selected True, Rank: 1.000
Column: 4, Selected True, Rank: 1.000
Column: 5, Selected True, Rank: 1.000
Column: 6, Selected False, Rank: 4.000
Column: 7, Selected False, Rank: 10.000
Column: 8, Selected False, Rank: 14.000
Column: 9, Selected True, Rank: 1.000
Column: 10, Selected False, Rank: 3.000
Column: 11, Selected False, Rank: 6.000
Column: 12, Selected True, Rank: 1.000
Column: 13, Selected False, Rank: 12.000
Column: 14, Selected False, Rank: 17.000
Column: 15, Selected False, Rank: 13.000
Column: 16, Selected False, Rank: 7.000
Column: 17, Selected False, Rank: 11.000
Column: 18, Selected False, Rank: 19.000
Column: 19, Selected False, Rank: 21.000
Column: 20, Selected False, Rank: 22.000
Column: 21, Selected False, Rank: 20.000
Column: 22, Selected True, Rank: 1.000
Column: 23, Selected False, Rank: 8.000
Column: 24, Selected True, Rank: 1.000
Column: 2

In [74]:
#ELM
input_size = rfe_X_train.shape[1]
hidden_size = 20

input_weights = np.random.uniform(-5,5,size=(input_size,hidden_size))
#input_weights = np.random.normal(size=[input_size, hidden_size])
biases = np.random.normal(size=[hidden_size])

print(input_weights,biases)


[[ 4.88805562  2.77138837  3.82140201  1.53286856 -4.0373123   1.93294358
   4.42434266  0.67100884  2.84122454 -1.67853258  3.01417301  3.28559293
   3.5456408  -3.52456046 -0.84186196 -2.423736   -2.9388649   2.03833621
   1.10475435  1.52290909]
 [ 3.42414244 -2.24627459 -4.379436    3.69002296  3.83645026  1.69869295
   4.09191768  3.89277587 -0.52499129  1.24281654  0.61631093  4.05870044
  -1.81927366 -0.23114241 -1.35901774  1.93343651  0.70439463 -4.02040211
  -1.10049114  1.68933115]
 [ 1.86327551  4.93233271  3.10073975 -3.96805109  2.92634767  2.70366203
   0.73965788 -2.88833691 -2.14573531  1.35087825 -2.84756993  4.68219106
  -1.75656306 -3.87632814 -3.30191715  3.24819717 -4.86845992 -1.81759128
   3.91516845  2.0554195 ]
 [ 2.49637459 -1.68192898  2.5900673  -4.07554631 -4.8989942   2.59489942
  -4.96834702  0.41816461 -1.30644059 -2.81437605 -1.72982826  0.55926077
  -1.90366401 -4.39915646 -2.91330427  0.26772979 -3.72672259  0.6979372
   1.45475515  2.43310956]
 [ 2.

In [75]:
def sigmoid(x):
    return 1.0 / (1.0 + np.exp(-x))

def hidden_nodes(X):
    G = np.dot(X, input_weights)
    G = G + biases
    H = sigmoid(G)
    return H

output_weights = np.dot(pinv2(hidden_nodes(rfe_X_train)), y_train)

In [76]:

def predict(X):
    out = hidden_nodes(X)
    out = np.dot(out, output_weights)
    return out

prediction = predict(rfe_X_test)
print (prediction, y_test)

[11.23109604  9.19589009  9.42753013 ...  9.1366453  12.04806404
 12.38017338] 0        16
1        16
2        16
3        16
4        20
         ..
22539    25
22540    16
22541    14
22542    20
22543    16
Name: labels, Length: 22544, dtype: int64


In [80]:

correct = 0
total = rfe_X_test.shape[0]
sum1 = 0

for i in range(total):
    predicted = prediction[i]
    actual = y_test[i]
    #print(prediction[i], predicted, y[i], actual)
    sum1+=abs((actual - predi
    print(sum1)

accuracy = sum1/total

print('Accuracy for ', hidden_size, ' hidden nodes: ', sum1)

nf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
i